In [3]:
# For a frequency, get aggregate bars for all stocks
import requests
import polygon
import pandas as pd
from datetime import datetime

client = polygon.RESTClient("NbOaYW2hvhGCEKNKaUeVI3iNg1P40Sie")
us_ticker_list_fpath = '/student/wangsaizhuo/Data/my_data_dir/main/us/instruments/all.txt'
with open(us_ticker_list_fpath) as f:
    us_ticker_list = f.read().splitlines()
us_stocks = [x.split('\t')[0] for x in us_ticker_list]



In [4]:
from datetime import timedelta


def get_aggs_for_all_stocks(freq, start, end):
    print(f"Getting {freq} bars for all stocks from {start} to {end}")
    all_aggs = {}

    def stock_worker(stock) -> pd.Series:
        client = polygon.RESTClient("NbOaYW2hvhGCEKNKaUeVI3iNg1P40Sie")
        total_aggs = []
        get_start = start
        get_end = end
        
        while True:
            aggs = client.get_aggs(
                ticker=stock, multiplier=1, timespan=freq, from_=get_start, to=end
            )
            total_aggs.extend(aggs)
            time_indices = [
                datetime.fromtimestamp(x.timestamp / 1000.0) for x in aggs
            ]
            if time_indices[-1].strftime("%Y-%m-%d") >= end:
                break
            get_start = time_indices[-1] + timedelta(minutes=1)
            print(f"{stock} starts at {get_start}")
        
        aggs = total_aggs
            
        # for each field, return a series
        time_indices = [
            datetime.fromtimestamp(x.timestamp / 1000.0) for x in aggs
        ]
        time_indices = [x.strftime("%Y%m%dT%H%M%S") for x in time_indices]
        open_prices = [x.open for x in aggs]
        high_prices = [x.high for x in aggs]
        low_prices = [x.low for x in aggs]
        close_prices = [x.close for x in aggs]
        volumes = [x.volume for x in aggs]
        vwaps = [x.vwap for x in aggs]
        # amounts = [x.volume * x.vwap for x in aggs]
        ntrades = [x.transactions for x in aggs]
        series_dict = {
            "open": pd.Series(open_prices, index=time_indices, name=stock),
            "close": pd.Series(close_prices, index=time_indices, name=stock),
            "high": pd.Series(high_prices, index=time_indices, name=stock),
            "low": pd.Series(low_prices, index=time_indices, name=stock),
            "volume": pd.Series(volumes, index=time_indices, name=stock),
            "vwap": pd.Series(vwaps, index=time_indices, name=stock),
            # "amount": pd.Series(amounts, index=time_indices, name=stock),
            "ntrades": pd.Series(ntrades, index=time_indices, name=stock),
        }
        df = pd.DataFrame(series_dict)
        df.to_csv(f'./{stock}.csv')
        # return df
        # return series_dict

    import loky
    from tqdm import tqdm

    with loky.get_reusable_executor(500) as e:
        all_series = [x for x in tqdm(e.map(stock_worker, us_stocks), total=len(us_stocks))]
        for i, series_dict in enumerate(all_series):
            print(f"Got {len(series_dict)} bars for {us_stocks[i]}")
            for k,v in series_dict.items():
                v.to_csv(f"./{us_stocks[i]}_{k}.csv")
                break
            break
        # # For each data field, make a dataframe with each column as a stock
        # for field in all_series[0].keys():
        #     all_aggs[field] = pd.concat([x[field] for x in all_series], axis=1)
    return all_series

In [5]:
aggs = get_aggs_for_all_stocks("minute", "2014-01-01", "2024-05-31")

Getting minute bars for all stocks from 2014-01-01 to 2024-05-31


  0%|          | 0/6374 [00:00<?, ?it/s]

A starts at 2014-01-22 04:52:00
AABA starts at 2017-07-06 23:16:00
AA starts at 2014-01-17 02:13:00
AABA starts at 2017-07-25 02:08:00
A starts at 2014-02-08 03:54:00
AA starts at 2014-01-31 21:34:00
AABA starts at 2017-08-10 21:46:00
AA starts at 2014-02-18 23:38:00
A starts at 2014-02-28 00:56:00
AABA starts at 2017-08-26 03:47:00
AA starts at 2014-03-06 04:32:00
AAC starts at 2014-11-26 01:40:00
AABA starts at 2017-09-14 23:06:00
AAME starts at 2015-05-28 22:09:00
AA starts at 2014-03-22 00:09:00
ABEO starts at 2015-07-17 21:49:00
A starts at 2014-03-18 23:14:00
AAL starts at 2014-01-18 02:01:00
AAPL starts at 2014-01-15 00:36:00
AAC starts at 2015-01-23 23:57:00
AAON starts at 2014-02-13 02:21:00
AAME starts at 2017-08-01 03:52:00
AAIC starts at 2020-12-08 04:22:00
ABUS starts at 2015-08-26 00:42:00
ACBAU starts at 2024-05-03 23:46:00
AB starts at 2014-01-31 00:33:00
AABA starts at 2017-10-03 01:13:00
AADI starts at 2021-11-03 02:24:00
ACAXU starts at 2024-01-09 05:00:00
ACAB start

  0%|          | 0/6374 [01:09<?, ?it/s]

APTX starts at 2018-12-29 04:58:00
ATIF starts at 2020-01-14 03:40:00
AMST starts at 2021-07-13 00:15:00
AKTX starts at 2017-04-25 22:09:00
AQST starts at 2019-01-08 04:58:00
ACER starts at 2019-05-29 02:33:00
AMTI starts at 2020-09-16 01:01:00


ASTE starts at 2014-07-08 23:48:00
ACRS starts at 2016-06-08 00:30:00
ACLX starts at 2022-09-28 02:26:00
ADXN starts at 2022-08-27 03:14:00
AMV starts at 2023-02-16 18:44:00
APA starts at 2014-03-18 03:32:00
ARDX starts at 2015-04-01 23:41:00
ALB starts at 2014-04-01 00:13:00
AMLX starts at 2022-02-04 03:08:00
ALKT starts at 2021-07-01 01:59:00
AOSL starts at 2014-10-24 03:13:00
ACHV starts at 2018-06-18 21:39:00
ATNM starts at 2014-10-23 00:51:00
AG starts at 2014-03-21 01:48:00
ATHX starts at 2014-02-12 03:40:00
ASYS starts at 2015-07-14 02:23:00
AUDC starts at 2014-03-06 00:00:00
AMC starts at 2014-06-21 02:51:00
AAT starts at 2014-10-10 02:10:00
ACST starts at 2015-02-28 02:19:00
AUID starts at 2021-11-17 01:26:00
ABCB starts at 2014-10-30 23:24:00
ARTW starts at 2021-07-27 22:18:00
APM starts at 2021-01-21 19:51:00
ATNX starts at 2017-12-13 00:19:00
AEL starts at 2014-05-08 23:23:00
AAWW starts at 2014-03-26 21:48:00
ALT starts at 2018-01-11 23:51:00
AMSF starts at 2014-03-04 22:5

In [ ]:
import os
day_vp_dir = '/student/wangsaizhuo/Data/my_data_dir/main/us/aggs'
files = os.listdir(day_vp_dir)
from tqdm import tqdm
data_dict = {}
for f in tqdm(files):
    if not f.endswith('.csv'):
        continue
    data = pd.read_csv(os.path.join(day_vp_dir, f), index_col=0)
    stock = f.split('.')[0]
    for col in data.columns:
        data_dict.setdefault(col, {})[stock] = data[col]